<a href="https://colab.research.google.com/github/gokulbytes/social-media-sentiment-analysis-for-brand-monitoring/blob/main/main_(4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requirements

In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

# Load Data

In [4]:
df=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=['Target','ID','Date','Flag','User','Text'],encoding='ISO-8859-1')

# Explore Data

In [5]:
df.shape

(1600000, 6)

In [6]:
df.head()

,Target,ID,Date,Flag,User,Text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [7]:
df.isnull().sum()

,0
Target,0
ID,0
Date,0
Flag,0
User,0
Text,0


In [8]:
df['Target'].value_counts()

# Negative_Tweeet=0
# Positive_Tweet=1

,count
Target,
0,800000
4,800000


# Data Transformation

In [9]:
df.replace({'Target':{4:1}},inplace=True)

## Stemming

In [10]:
port_stem=PorterStemmer()

In [11]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [12]:
df['Stemmed_Content']=df['Text'].apply(stemming)

## Features & Target

In [13]:
x=df['Stemmed_Content'].values
y=df['Target'].values

In [16]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=2)

# Vectorization

In [17]:
vecorizer = TfidfVectorizer()

X_train = vecorizer.fit_transform(X_train)
X_test = vecorizer.transform(X_test)

# Logistic Regression

In [18]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

# Model Evaluation

In [19]:
x_train_prediction = model.predict(X_train)
train_accuracy = accuracy_score(y_train, x_train_prediction)
print('Accuracy on training data: ', train_accuracy)

Accuracy on training data:  0.79871953125


In [20]:
x_test_prediction = model.predict(X_test)
test_accuracy = accuracy_score(y_test, x_test_prediction)
print('Accuracy on test data: ', test_accuracy)

Accuracy on test data:  0.77668125


# Save Model

In [22]:
import joblib

joblib.dump(model,'model.pkl')
joblib.dump(vecorizer,'vectorizer.pkl')

['vectorizer.pkl']